# Support Vector Regression - Experimento

Este componente treina um modelo Support Vector Regression usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê diversas ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parameters
dataset = "boston" #@param {type:"string"}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# features to apply One Hot Encoder
one_hot_features = "" #@param {type:"feature",multiple:true,label:"Features para fazer codificação one-hot", description: "Seu modelo utilizará a codificação one-hot para as features selecionadas. As demais features categóricas serão codificadas utilizando a codificação ordinal."}

# hyperparameters
kernel = "rbf" #@param ["linear", "poly", "rbf", "sigmoid", "precomputed"] {type:"string", label:"Kernel", description:"Especifica o tipo de kernel a ser usado no algoritmo"}
degree = 3 #@param {type:"integer", label:"Grau", description:"Grau da função polinomial do kernel ('poly'). Ignorado por outros kernels"}
gamma = "auto" #@param ["scale", "auto"] {type: "string", label:"Gama", description:"Coeficiente de kernel para 'rbf', 'poly' e 'sigmoid'"} 
C = 1.0 #@param {type:"number", label:"Regularização", description:"A força da regularização é inversamente proporcional a C. Deve ser positivo. Penalidade é l2²"}
max_iter = -1 #@param {type:"integer", label:"Iteração", description:"Limite fixo nas iterações no solver, ou -1 sem limite"}

## Acesso ao conjunto de dados

Utiliza a função `load_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar conjuntos de dados.
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Divide dataset em subconjuntos de treino e teste

Subconjunto de treino: amostra de dados usada para treinar o modelo.<br>
Subconjunto de teste: amostra de dados usada para fornecer uma avaliação imparcial do treinamento do modelo no subconjunto de dados de treino.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.7)

## Configuração dos atributos

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]


# Selects non-numerical features to apply ordinal encoder or one-hot encoder
one_hot_features = np.asarray(one_hot_features)
non_numerical_indexes_one_hot = np.where(~(featuretypes == NUMERICAL) & np.isin(columns,one_hot_features))[0]
non_numerical_indexes_ordinal = np.where(~(featuretypes == NUMERICAL) & ~(np.isin(columns,one_hot_features)))[0]


# After the step handle_missing_values, 
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes))
non_numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes), len(featuretypes))
one_hot_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[np.where(np.isin(non_numerical_indexes,non_numerical_indexes_one_hot))[0]]         
ordinal_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[np.where(np.isin(non_numerical_indexes,non_numerical_indexes_ordinal))[0]]                                              

## Treina modelo usando sklearn.svm.SVR

In [ ]:
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[
    ('handle_missing_values',
     ColumnTransformer(
        [('imputer_mean', SimpleImputer(strategy='mean'), numerical_indexes),
         ('imputer_mode', SimpleImputer(strategy='most_frequent'), non_numerical_indexes)],
         remainder='drop')),
    ('handle_categorical_features',
    ColumnTransformer(
         [('feature_encoder_ordinal', OrdinalEncoder(), ordinal_indexes_after_handle_missing_values),
          ('feature_encoder_onehot', OneHotEncoder(), one_hot_indexes_after_handle_missing_values)],
         remainder='passthrough')),
    ('estimator', SVR(kernel=kernel,
                      degree=degree,
                      gamma=gamma,
                      C=C,
                      max_iter=max_iter))
])

pipeline.fit(X_train, y_train)    

## Avaliação de desempenho

O [Coeficiente de determinação](https://pt.wikipedia.org/wiki/Coeficiente_de_determinação) (ou R²), corresponde à correlação ao quadrado entre os valores de resultado observados e os valores previstos pelo modelo.

In [ ]:
from sklearn.metrics import r2_score

# uses the model to make predictions on the Test Dataset
y_pred = pipeline.predict(X_test)

# computes R²
r2 = r2_score(y_test, y_pred)
pipeline.fit(X, y)

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
from platiagro import save_metrics

save_metrics(r2_score=r2)

## Salva figuras

Utiliza a função `save_figures` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar figuras do [matplotlib](https://matplotlib.org/3.2.1/gallery/index.html). <br>
Usa-se a [Curva ROC (ROC)](https://pt.wikipedia.org/wiki/Caracter%C3%ADstica_de_Opera%C3%A7%C3%A3o_do_Receptor) para avaliar o desempenho do modelo. Ilustra-se o desempenho de um sistema classificador binário, pois seu limite de discriminação é variado. <br>
Para a tarefa de classificação em várias classes, é utilizado o algoritmo one-vs-rest, ou seja, calcula a AUC de cada classe em relação ao restante.

In [ ]:
import matplotlib.pyplot as plt
from platiagro import save_figure
from scipy.stats import gaussian_kde


def annotate_plot(e, s, plt, y_lim, h, abs_err):
    if h < 2:
        p = 0.05
    else:
        p = 0.1
    plt.annotate("", xy=(max(e), y_lim[1]/h),
                 xytext=(0, y_lim[1]/h),
                 arrowprops=dict(arrowstyle="->"))
    plt.annotate("", xy=(min(e), y_lim[1]/h),
                 xytext=(0, y_lim[1]/h),
                 arrowprops=dict(arrowstyle="->"))
    plt.annotate("{}%".format(s),
                 xy=(0, (1+p)*y_lim[1]/h),
                 ha="center")
    if abs_err:
        plt.annotate("{:.2f}".format(max(e)),
                     xy=((0+max(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")
        plt.annotate("{:.2f}".format(min(e)),
                     xy=((0+min(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")
    else:
        plt.annotate("{:.2f}%".format(100*max(e)),
                     xy=((0+max(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")
        plt.annotate("{:.2f}%".format(100*min(e)),
                     xy=((0+min(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")

In [ ]:
abs_err = False
if any(y_test==0):
    err = y_pred - y_test
    abs_err = True
else:
    err = (y_pred - y_test)/y_test

sorted_idx = np.argsort(np.abs(err))
n = int(0.7*len(y_test))
idx = sorted_idx[:n]
e = err[idx]

n = int(0.95*len(y_test))
idx = sorted_idx[:n]
aux = err[idx]
x_lim = (aux.min(), aux.max())

plt.figure()

kde = gaussian_kde(err)
x_err = np.linspace(err.min(), err.max(), 1000)
p_err = kde(x_err)
plt.plot(x_err, p_err, 'b-')

y_lim = plt.ylim()
plt.ylim((0, y_lim[1]))
y_lim = plt.ylim()
plt.xlim(x_lim)
plt.plot([e.min(), e.min()], y_lim, "r--")
plt.plot([e.max(), e.max()], y_lim, "r--")

# Shade the area between e.min() and e.max()
plt.fill_betweenx(y_lim, e.min(), e.max(),
                  facecolor="red",  # The fill color
                  color="red",      # The outline color
                  alpha=0.2)        # Transparency of the fill

annotate_plot(e, 70, plt, y_lim, 2, abs_err)
annotate_plot(aux, 95, plt, y_lim, 1.2, abs_err)

plt.grid(True)
plt.title("Error Distribution")

save_figure(figure=plt.gcf())

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline, columns=columns)